In [5]:
import pandas as pd
import geopandas as gpd
import numpy as np
from datetime import datetime, timedelta

In [6]:
# OPTION 1: Load from pre-extracted file (recommended for low memory systems)
# First run: python extract_recent_data.py
# Then uncomment this section and comment out Option 2

# print('Loading pre-extracted data...')
# crime_df = pd.read_csv('../data/NYPD_data_2020plus.csv', parse_dates=['CMPLNT_FR_DT'])
# crime_df = crime_df.rename(columns={
#     'CMPLNT_NUM': 'ID',
#     'CMPLNT_FR_DT': 'Date',
#     'OFNS_DESC': 'Primary Type',
#     'X_COORD_CD': 'x',
#     'Y_COORD_CD': 'y'
# })
# crime_df['Year'] = crime_df['Date'].dt.year
# crime_df['Date'] = crime_df['Date'].dt.date
# print(f"Loaded {len(crime_df):,} records from {crime_df['Date'].min()} to {crime_df['Date'].max()}")

# OPTION 2: Load directly from large file (may cause memory errors)
print('Loading NYPD crime data directly (this may take a few minutes)...')

cols_to_use = ['CMPLNT_NUM', 'CMPLNT_FR_DT', 'OFNS_DESC', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude']
chunk_size = 50000
chunks = []
total_processed = 0
total_kept = 0

for i, chunk in enumerate(pd.read_csv('../../NYPD_data.csv', usecols=cols_to_use, chunksize=chunk_size, low_memory=False)):
    total_processed += len(chunk)
    
    # Parse dates efficiently
    chunk['CMPLNT_FR_DT'] = pd.to_datetime(chunk['CMPLNT_FR_DT'], format='%m/%d/%Y', errors='coerce')
    
    # Filter immediately to reduce memory
    chunk = chunk.dropna(subset=['CMPLNT_FR_DT'])
    chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
    chunk = chunk[chunk['Year'] >= 2020]
    
    # Clean coordinates immediately
    chunk = chunk.dropna(subset=['X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude'])
    chunk = chunk[(chunk['X_COORD_CD'] != 0) & (chunk['Y_COORD_CD'] != 0)]
    chunk = chunk[(chunk['Latitude'] != 0) & (chunk['Longitude'] != 0)]
    
    if len(chunk) > 0:
        total_kept += len(chunk)
        chunks.append(chunk)
    
    if (i + 1) % 10 == 0:
        print(f'Processed {total_processed:,} rows, kept {total_kept:,} records (chunk {i+1})...')

print(f'\nTotal processed: {total_processed:,} rows, kept {total_kept:,} records')

if chunks:
    crime_df = pd.concat(chunks, ignore_index=True)
    crime_df = crime_df.rename(columns={
        'CMPLNT_NUM': 'ID',
        'CMPLNT_FR_DT': 'Date',
        'OFNS_DESC': 'Primary Type',
        'X_COORD_CD': 'x',
        'Y_COORD_CD': 'y'
    })
    crime_df['Date'] = crime_df['Date'].dt.date
    
    print(f"\nFinal dataset: {len(crime_df):,} crime records")
    print(f"Date range: {crime_df['Date'].min()} to {crime_df['Date'].max()}")
    print(f"Years included: {sorted(crime_df['Year'].unique())}")
    print(f"Memory usage: {crime_df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
else:
    print("ERROR: No data was loaded!")

Loading NYPD crime data directly (this may take a few minutes)...


C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

Processed 500,000 rows, kept 499,011 records (chunk 10)...


C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

Processed 1,000,000 rows, kept 997,851 records (chunk 20)...


...

Processed 8,500,000 rows, kept 2,512,496 records (chunk 170)...


C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

Processed 9,000,000 rows, kept 2,512,496 records (chunk 180)...


C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk['Year'] = chunk['CMPLNT_FR_DT'].dt.year
C:\Users\Fradi\AppData\Local\Temp\ipykernel_9096\2959968878.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

Processed 9,491,946 rows, kept 2,512,496 records (chunk 190)...

Total processed: 9,491,946 rows, kept 2,512,496 records

Final dataset: 2,512,496 crime records
Date range: 2020-01-01 to 2024-12-31
Years included: [2020, 2021, 2022, 2023, 2024]
Memory usage: 714.82 MB


In [7]:
# Create grid cells based on x and y coordinates
# Note: You'll need to download NYC borough shapefiles
# path = '../data/shapefile/nyc_boroughs.shp'
# nyc = gpd.read_file(path)
# nyc = nyc.dissolve()

# For now, using bounding box from data
xmin, xmax = crime_df['Longitude'].min(), crime_df['Longitude'].max()
ymin, ymax = crime_df['Latitude'].min(), crime_df['Latitude'].max()

n_x_cells = 50
x_cell_size = (xmax - xmin) / n_x_cells
n_y_cells = round(((xmax - xmin)/(ymax - ymin))*n_x_cells)
y_cell_size = (ymax - ymin) / n_y_cells

# Determine grid cell coordinate based on x and y coordinates
crime_df['x_cell'] = ((crime_df['Longitude'] - xmin) // x_cell_size).astype(int)
crime_df['y_cell'] = n_y_cells - ((crime_df['Latitude'] - ymin) // y_cell_size).astype(int) - 1

# Clip to valid range
crime_df['x_cell'] = crime_df['x_cell'].clip(0, n_x_cells - 1)
crime_df['y_cell'] = crime_df['y_cell'].clip(0, n_y_cells - 1)

crime_df['x_y_cell'] = list(zip(crime_df['x_cell'], crime_df['y_cell']))

print(f"Grid dimensions: {n_y_cells} x {n_x_cells}")

Grid dimensions: 67 x 50


In [8]:
display(crime_df.head())

,ID,Date,Primary Type,x,y,Latitude,Longitude,Year,x_cell,y_cell,x_y_cell
0,298697963,2024-12-30,PETIT LARCENY,"988,579","191,102",40.691215,-73.984388,2024,24,35,"(24, 35)"
1,298704204,2024-12-31,SEX CRIMES,"988,886","207,857",40.737203,-73.983273,2024,24,28,"(24, 28)"
2,298704254,2024-12-31,DANGEROUS DRUGS,"975,608","163,701",40.615994,-74.031128,2024,20,48,"(20, 48)"
3,298687089,2024-12-27,CRIMINAL MISCHIEF & RELATED OF,"999,875","218,955",40.767654,-73.943590,2024,28,23,"(28, 23)"
4,298695167,2024-12-30,OFF. AGNST PUB ORD SENSBLTY &,"983,756","160,323",40.606736,-74.001777,2024,22,49,"(22, 49)"


In [9]:
# Crime severity classification for NYC crime types
print(crime_df['Primary Type'].unique())

# Low severity crimes
soft_list = [
    'ADMINISTRATIVE CODE', 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 'ALCOHOLIC BEVERAGE CONTROL LAW',
    'ANTICIPATORY OFFENSES', 'DISORDERLY CONDUCT', 'ENDAN WELFARE INCOMP',
    'GAMBLING', 'JOSTLING', 'LOITERING', 'LOITERING/GAMBLING (CARDS, DIC',
    'NEW YORK CITY HEALTH CODE', 'OFF. AGNST PUB ORD SENSBLTY &', 'OTHER STATE LAWS',
    'OTHER STATE LAWS (NON PENAL LAW)', 'PROSTITUTION & RELATED OFFENSES',
    'PUBLIC ADMINISTRATION', 'UNAUTHORIZED USE OF A VEHICLE 3 (UUV)'
]
soft_list = list(zip(soft_list, [0]*len(soft_list)))

# Medium severity crimes  
middle_list = [
    'ARSON', 'CRIMINAL MISCHIEF & RELATED OF', 'CRIMINAL TRESPASS',
    'FORGERY', 'FRAUDS', 'FRAUDULENT ACCOSTING',
    'HARASSMENT', 'INTOXICATED & IMPAIRED DRIVING', 'INTOXICATED/IMPAIRED DRIVING',
    'MISCELLANEOUS PENAL LAW', 'NYS LAWS-UNCLASSIFIED FELONY', 'NYS LAWS-UNCLASSIFIED VIOLATION',
    'OFFENSES AGAINST PUBLIC ADMINI', 'OFFENSES AGAINST PUBLIC SAFETY',
    'OFFENSES AGAINST THE PERSON', 'OFFENSES INVOLVING FRAUD',
    'OTHER OFFENSES RELATED TO THEF', 'POSSESSION OF STOLEN PROPERTY',
    'THEFT-FRAUD', 'VEHICLE AND TRAFFIC LAWS'
]
middle_list = list(zip(middle_list, [0.5]*len(middle_list)))

# High severity crimes
bad_list = [
    'ASSAULT 3 & RELATED OFFENSES', 'BURGLARY', 'DANGEROUS DRUGS', 'DANGEROUS WEAPONS',
    'FELONY ASSAULT', 'FELONY SEX CRIMES', 'GRAND LARCENY', 'GRAND LARCENY OF MOTOR VEHICLE',
    'HARRASSMENT 2', 'KIDNAPPING', 'KIDNAPPING & RELATED OFFENSES',
    'MURDER & NON-NEGL. MANSLAUGHTER', 'MURDER & NON-NEGL. MANSLAUGHTE', 'NEGLIGENT HOMICIDE',
    'OFFENSES RELATED TO CHILDREN', 'PETIT LARCENY', 'RAPE', 'ROBBERY',
    'SEX CRIMES', 'SEXUAL ABUSE', 'THEFT OF SERVICES', 'FELONY THEFT',
    'HOMICIDE-NEGLIGENT,UNCLASSIFIE', 'HOMICIDE-NEGLIGENT-VEHICLE',
    'KIDNAPPING AND RELATED OFFENSES', 'OTHER TRAFFIC INFRACTION'
]
bad_list = list(zip(bad_list, [1]*len(bad_list)))

score_dict = dict(soft_list + middle_list + bad_list)

# For crimes not in our classification, assign medium severity
test_crime_df = crime_df.copy()
test_crime_df['Primary Type'] = test_crime_df['Primary Type'].map(lambda x: score_dict.get(x, 0.5))

print(f"\nSeverity distribution:")
print(test_crime_df['Primary Type'].value_counts())

['PETIT LARCENY' 'SEX CRIMES' 'DANGEROUS DRUGS'
 'CRIMINAL MISCHIEF & RELATED OF' 'OFF. AGNST PUB ORD SENSBLTY &'
 'JOSTLING' 'ASSAULT 3 & RELATED OFFENSES' 'RAPE' 'ROBBERY'
 'OTHER STATE LAWS' 'FORGERY' 'CRIMINAL TRESPASS' 'HARRASSMENT 2'
 'VEHICLE AND TRAFFIC LAWS' 'GRAND LARCENY OF MOTOR VEHICLE'
 'GRAND LARCENY' 'POSSESSION OF STOLEN PROPERTY' 'FELONY ASSAULT'
 'MISCELLANEOUS PENAL LAW' 'OFFENSES AGAINST PUBLIC ADMINI' 'BURGLARY'
 'INTOXICATED & IMPAIRED DRIVING' 'OFFENSES INVOLVING FRAUD' 'THEFT-FRAUD'
 'OFFENSES AGAINST THE PERSON' 'DANGEROUS WEAPONS'
 'OTHER STATE LAWS (NON PENAL LAW)' 'UNAUTHORIZED USE OF A VEHICLE'
 'MURDER & NON-NEGL. MANSLAUGHTER' "BURGLAR'S TOOLS"
 'OTHER OFFENSES RELATED TO THEFT' 'PETIT LARCENY OF MOTOR VEHICLE'
 'CANNABIS RELATED OFFENSES' 'FRAUDS' 'ARSON' 'ADMINISTRATIVE CODE'
 'ALCOHOLIC BEVERAGE CONTROL LAW' 'CHILD ABANDONMENT/NON SUPPORT 1'
 'GAMBLING' 'KIDNAPPING & RELATED OFFENSES'
 'PROSTITUTION & RELATED OFFENSES' 'OFFENSES AGAINST PUBLIC SAFETY'

In [10]:
# Group crime data by date and grid cell
grouped_crimes_df = test_crime_df.groupby(['Date','x_y_cell']).agg(
    {
        'Primary Type': 'mean',
        'ID': 'count'
    }).reset_index()
grouped_crimes_df = grouped_crimes_df.rename(columns={'ID':'Count'})

grouped_crimes_df[['x_cell', 'y_cell']] = grouped_crimes_df['x_y_cell'].to_list()

display(grouped_crimes_df)

,Date,x_y_cell,Primary Type,Count,x_cell,y_cell
0,2020-01-01,"(1, 60)",0.5,1,1,60
1,2020-01-01,"(1, 62)",1.0,1,1,62
2,2020-01-01,"(1, 65)",0.5,1,1,65
3,2020-01-01,"(5, 52)",0.5,1,5,52
4,2020-01-01,"(5, 57)",1.0,1,5,57
...,...,...,...,...,...,...
1012498,2024-12-31,"(46, 35)",1.0,1,46,35
1012499,2024-12-31,"(46, 38)",0.5,1,46,38
1012500,2024-12-31,"(46, 51)",1.0,1,46,51
1012501,2024-12-31,"(47, 32)",0.5,2,47,32


In [11]:
# Check for missing days
deltas = grouped_crimes_df['Date'].diff()[1:]
gaps = deltas[deltas > timedelta(days=1)]
print(f'{len(gaps)} gaps with average gap duration: {gaps.mean()}')

0 gaps with average gap duration: NaT


Shape input matrix: (n_days, n_y_cells, n_x_cells) - using Count as feature

In [12]:
# Create daily crime count arrays
day_group_crimes_df = grouped_crimes_df.groupby('Date')
all_days = []
for day_df in day_group_crimes_df:
    day_array = np.zeros((n_y_cells, n_x_cells))
    for index, row in day_df[1].iterrows():
        day_array[row['y_cell'], row['x_cell']] = row['Count']
    all_days.append(list(day_array))
all_days = np.array(all_days)

print(f"Crime data shape: {all_days.shape}")

Crime data shape: (1827, 67, 50)


In [13]:
# Save train and test data (last 365 days for testing)
np.save('../data/train_data.npy', all_days[:-365])
np.save('../data/test_data.npy', all_days[-365:])

print(f"Training data: {all_days[:-365].shape}")
print(f"Testing data: {all_days[-365:].shape}")

Training data: (1462, 67, 50)
Testing data: (365, 67, 50)


In [14]:
# Create daily crime severity arrays
all_days_severity = []
for day_df in day_group_crimes_df:
    day_array = np.zeros((n_y_cells, n_x_cells))
    for index, row in day_df[1].iterrows():
        day_array[row['y_cell'], row['x_cell']] = row['Primary Type']
    all_days_severity.append(list(day_array))
all_days_severity = np.array(all_days_severity)

In [15]:
# Save severity data
np.save('../data/train_data_severity.npy', all_days_severity[:-365])
np.save('../data/test_data_severity.npy', all_days_severity[-365:])

### Crime seasonality

In [16]:
# Create seasonality data (day of year)
dates = [item.timetuple().tm_yday for item, _ in test_crime_df.groupby('Date')]
dates2 = []
for val in dates:
    dates2.append(np.tile(val, (n_y_cells, n_x_cells)))
dates2 = np.array(dates2)

np.save('../data/train_data_seasonality.npy', dates2[:-365])
np.save('../data/test_data_seasonality.npy', dates2[-365:])